# Playing with generator

In [1]:
#%%timeit
import glob
import os
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import torch
import numpy as np
import sys
import copy
from pathlib import Path
#from src.envs.generators import RandomRecurrence
#from src.envs.encoders import Equation, IntegerSeries, RealSeries
from sympy import *
import pickle
from collections import defaultdict, OrderedDict
import math

%reload_ext autoreload

#%cd ~/recur/notebooks
sys.path.append('/private/home/pakamienny/Research_2/recur')
import src
from src.envs.generators import RandomRecurrence
from src.envs.encoders import Equation, RealSeries, IntegerSeries

class params:
    float_precision = 3
    max_len = 30
    max_ops = 6
    max_degree = 6
    max_number = 1e100
    real_series = False
    output_numeric=True
    max_exponent = 100
    init_scale = 10
    prob_const=1/3
    prob_n = 1/3
    max_int = 10
    prob_rand = 0.2
    dimension=5
    int_base=10
    n_predictions=5

In [2]:
generator = RandomRecurrence(params)

rng = np.random.RandomState(0)
import time 

SPECIAL_WORDS = ["EOS", "PAD", "(", ")", "SPECIAL"]

if params.real_series:
    input_encoder = RealSeries(params)
else:
    input_encoder = IntegerSeries(params)
input_words = SPECIAL_WORDS+sorted(list(set(input_encoder.symbols)))

if params.output_numeric:
    if params.real_series:
        output_encoder = RealSeries(params)
    else:
        output_encoder = IntegerSeries(params)
else:
    output_encoder = Equation(params)

deb = time.time()
k_trials=1000
successful_trials=0
size_input=[]
size_output=[]
for k in range(k_trials):
    tree, series, series_to_predict = generator.generate(rng, length=15, nb_ops = 1, prediction_points=True)
    if tree is not None:
        successful_trials+=1
        size_input.append(len(input_encoder.encode(series)))
        if not params.output_numeric:
            size_output.append(len(output_encoder.encode(tree)))
        else:
            size_output.append(len(output_encoder.encode(series_to_predict)))

        #print(tree)
        #print(series)
        #for i in range(params.dimension):
        #    plt.title(tree.infix())
        #    plt.plot(series[i::params.dimension])
        #plt.yscale('symlog')
        #plt.show()
size_input=np.mean(np.array(size_input))
size_output=np.mean(np.array(size_output))

print("Percentage of successful generation: {}".format(successful_trials/k_trials*100))
print("Time: {}".format(time.time()-deb))
print("Average sizes: input {}, output {}".format(size_input.max(), size_output.max()))

/private/home/pakamienny/Research_2/recur/src/envs/generators.py:136: RuntimeWarning: divide by zero encountered in long_scalars
  else: return self.children[0].val(series) // self.children[1].val(series)
/private/home/pakamienny/Research_2/recur/src/envs/generators.py:130: RuntimeWarning: divide by zero encountered in long_scalars
  else: return self.children[0].val(series) % self.children[1].val(series)
/private/home/pakamienny/Research_2/recur/src/envs/generators.py:121: RuntimeWarning: overflow encountered in long_scalars
  return self.children[0].val(series) * self.children[1].val(series)


Percentage of successful generation: 65.8
Time: 1.6271309852600098
Average sizes: input 186.17325227963525, output 72.65957446808511


In [3]:
print(input_encoder.decode(["-",  "5"]))

[-5]


# Checking encoder

In [5]:
class params:
    int_base = 10
    has_separator = False
    
encoder = Equation(params)
print(tree.infix())
encoded = encoder.encode(tree)
print(encoded)
decoded = encoder.decode(encoded)
print(decoded.infix())

(8 mod n) | (rand)**2 | (rand mod n) | (8 add x_1_5) | abs(n)
['mod', '8', 'n', '|', 'sqr', 'rand', '|', 'mod', 'rand', 'n', '|', 'add', '8', 'x_1_5', '|', 'abs', 'n']
(8 mod n) | (rand)**2 | (rand mod n) | (8 add x_1_5) | abs(n)


In [6]:
class params:
    a=1
    
from src.envs.encoders import RealSeries

encoder = RealSeries(params)
x = series
print(x)
encoded = encoder.encode(x)
print(encoded)
decoded = encoder.decode(encoded)
print(decoded)

AttributeError: type object 'params' has no attribute 'float_precision'